In [1]:
from dataInput import *
from CV import *
from TwoStageSP import *
from FA import *
import numpy as np
import pandas as pd

In [2]:
inputParams = inputParams(1,1,57,1000);
solveParams = solveParams(100000,200,100000,10800,1e-4);

In [3]:
intensityFile = 'data/synthetic/intensity.csv';
locationFile = 'data/synthetic/location.csv';
landfallFile = 'data/synthetic/landfall_7.csv';

In [4]:
hurricaneDataSet = hurricaneInput(intensityFile, locationFile, landfallFile, inputParams)

In [5]:
netNodesFile = 'data/synthetic/nodes.csv';
netParamsFile = 'data/synthetic/netParams.csv';
networkDataSet = networkInput(3,20,0.05,netNodesFile,netParamsFile,hurricaneDataSet)

In [6]:
osfname = "./data/synthetic/OOS" + str(inputParams.k_init) + ".csv"
OS_paths = pd.read_csv(osfname).values  # Read the out-of-sample file

In [7]:
#static_2SSP_eval(networkDataSet, hurricaneDataSet, inputParams, solveParams, osfname)

In [8]:
RH_2SSP_eval(networkDataSet, hurricaneDataSet, inputParams, solveParams, osfname)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-01-04
RH 2SSP....
μ ± 1.96*σ/√NS = 7565.364754343242 ± 162.45279579505237


[array([[ 6649.81882093,   263.15132209,   263.15132209, ...,
          -959.81902483,     0.        ,     0.        ],
        [ 6649.81882093,   263.15132209,   263.15132209, ...,
           263.15132209, -1052.60528837,     0.        ],
        [ 6649.81882093,   263.15132209,   263.15132209, ...,
         -1052.60528837,     0.        ,     0.        ],
        ...,
        [ 6649.81882093,  7530.0615276 ,   526.74206307, ...,
         -1841.21904314,     0.        ,     0.        ],
        [ 6649.81882093,   263.15132209,   263.15132209, ...,
             0.        ,     0.        ,     0.        ],
        [ 6649.81882093,   263.15132209,   263.15132209, ...,
          -794.21200367,     0.        ,     0.        ]]),
 7565.364754343242,
 7402.911958548189,
 7727.817550138294,
 338.109384059906]